In [1]:
# 1. Kütüphane importları
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, hamming_loss, f1_score
import time
import warnings
import joblib
from multiprocessing import cpu_count
from joblib import Parallel, delayed, parallel_backend
from tqdm import tqdm
import os

In [2]:
# Sonuçların kaydedileceği dizini oluştur
output_dir = "C:\\Users\\DangerClose\\Desktop\\Stack Overflow Q&A\\active_learning_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# Model dizinini oluştur
models_dir = os.path.join(output_dir, "models")
if not os.path.exists(models_dir):
    os.makedirs(models_dir)


In [4]:
# Görseller dizinini oluştur
images_dir = os.path.join(output_dir, "images")
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

In [5]:
# Tüm uyarıları bastır
warnings.filterwarnings("ignore")

In [6]:
# Mevcut CPU çekirdek sayısını al
n_cores = cpu_count()
print(f"Kullanılabilir CPU çekirdek sayısı: {n_cores}")

Kullanılabilir CPU çekirdek sayısı: 12


In [7]:
# 1. Ön işlenmiş veriyi yükle
print("1. İşlenmiş veri yükleniyor...")
data_path = "C:\\Users\\DangerClose\\Desktop\\Stack Overflow Q&A\\model_ready_sample.csv"
data = pd.read_csv(data_path)
print(f"Veri yüklendi. Toplam örnek sayısı: {data.shape[0]}")

1. İşlenmiş veri yükleniyor...
Veri yüklendi. Toplam örnek sayısı: 20000


In [8]:
# 2. Etiketli ve etiketsiz verileri belirle
print("\n2. Etiketsiz sorular filtreleniyor...")
tag_columns = [col for col in data.columns if col.startswith('tag_')]
data_with_tags = data[data[tag_columns].sum(axis=1) > 0]
print(f"Etiketli sorular: {len(data_with_tags)} ({len(data_with_tags)/len(data)*100:.2f}%)")

# Etiket dağılımını görselleştir
print("\n3. Etiket dağılımı analiz ediliyor...")
tag_counts = data_with_tags[tag_columns].sum().sort_values(ascending=False)
top_tags = tag_counts.head(20)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_tags.values, y=top_tags.index)
plt.title('En Sık Kullanılan 20 Etiket')
plt.xlabel('Soru Sayısı')
plt.ylabel('Etiketler')
plt.tight_layout()
plt.savefig(os.path.join(images_dir, 'top_tags_distribution.png'))
plt.close()


2. Etiketsiz sorular filtreleniyor...
Etiketli sorular: 13595 (67.97%)

3. Etiket dağılımı analiz ediliyor...


In [9]:
# 3. Veriyi hazırla
print("\n4. Veri seti hazırlanıyor...")
X_text = data_with_tags['complete_text']  # Ana metin özelliği
y = data_with_tags[tag_columns].values

# Eğitim ve test setlerine ayır
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, 
    stratify=data_with_tags[tag_columns].sum(axis=1) > 1
)
print(f"Eğitim seti: {X_train_text.shape[0]} örnek")
print(f"Test seti: {X_test_text.shape[0]} örnek")



4. Veri seti hazırlanıyor...
Eğitim seti: 10876 örnek
Test seti: 2719 örnek


In [10]:
# 4. TF-IDF vektörleştirme (paralelleştirme ile)
print("\n5. Metin vektörleştirme (TF-IDF)...")
tfidf_vectorizer = TfidfVectorizer(
    max_features=15000,
    min_df=3,
    max_df=0.85,
    ngram_range=(1, 2),
    sublinear_tf=True,
    stop_words='english'
)

# Vektörleştirme işlemlerini paralelleştirme
with parallel_backend('threading', n_jobs=n_cores):
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
    X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

print(f"Özellik sayısı: {X_train_tfidf.shape[1]}")

# TF-IDF özelliklerini görselleştir
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_sorting = np.argsort(tfidf_vectorizer.idf_)[::-1]
top_features = [feature_names[i] for i in tfidf_sorting[:20]]
top_idf_scores = [tfidf_vectorizer.idf_[i] for i in tfidf_sorting[:20]]

plt.figure(figsize=(12, 8))
sns.barplot(x=top_idf_scores, y=top_features)
plt.title('En Yüksek TF-IDF Skoru Olan 20 Özellik')
plt.xlabel('TF-IDF Skoru')
plt.ylabel('Özellikler')
plt.tight_layout()
plt.savefig(os.path.join(images_dir, 'top_tfidf_features.png'))
plt.close()

# Ara sonuçları kaydet (hesaplamaları tekrarlamayı önlemek için)
print("Vektörleştirilmiş verileri diske kaydediyorum...")
joblib.dump(X_train_tfidf, os.path.join(models_dir, 'X_train_tfidf.joblib'))
joblib.dump(X_test_tfidf, os.path.join(models_dir, 'X_test_tfidf.joblib'))
joblib.dump(y_train, os.path.join(models_dir, 'y_train.joblib'))
joblib.dump(y_test, os.path.join(models_dir, 'y_test.joblib'))
print("Vektörleştirilmiş veriler başarıyla kaydedildi.")


5. Metin vektörleştirme (TF-IDF)...
Özellik sayısı: 15000
Vektörleştirilmiş verileri diske kaydediyorum...
Vektörleştirilmiş veriler başarıyla kaydedildi.


In [11]:
# 5. Aktif öğrenme için eğitim setinden başlangıç ve havuz verileri oluştur
n_initial = 2000  # Başlangıçta kullanılacak etiketli örnek sayısı

# Stratifiye başlangıç örnek seçimi (paralel işleme ile)
print("\n6. Başlangıç veri seti oluşturuluyor...")
etiket_sayilari = y_train.sum(axis=1)
az_etiketli = np.where(etiket_sayilari == 1)[0]
cok_etiketli = np.where(etiket_sayilari > 1)[0]

# Az etiketli ve çok etiketli örnekleri dengeli şekilde seçelim
n_az_etiketli = min(len(az_etiketli), n_initial // 2)
n_cok_etiketli = min(len(cok_etiketli), n_initial - n_az_etiketli)

# İndeksleri belirle
np.random.seed(42)  # Tekrarlanabilirlik için
initial_idx_az = np.random.choice(az_etiketli, size=n_az_etiketli, replace=False)
initial_idx_cok = np.random.choice(cok_etiketli, size=n_cok_etiketli, replace=False)
initial_idx = np.concatenate([initial_idx_az, initial_idx_cok])

# Başlangıç verileri
X_initial = X_train_tfidf[initial_idx]
y_initial = y_train[initial_idx]

# Havuz için geriye kalan örnekleri ayır
pool_idx = np.setdiff1d(range(X_train_tfidf.shape[0]), initial_idx)
X_pool = X_train_tfidf[pool_idx]
y_pool = y_train[pool_idx]

print(f"Başlangıç seti: {len(initial_idx)} örnek ({n_az_etiketli} az etiketli, {n_cok_etiketli} çok etiketli)")
print(f"Havuz boyutu: {len(pool_idx)} örnek")


6. Başlangıç veri seti oluşturuluyor...
Başlangıç seti: 2000 örnek (1000 az etiketli, 1000 çok etiketli)
Havuz boyutu: 8876 örnek


In [12]:
# 6. Base classifier tanımla (tüm çekirdekleri kullanarak)
print("\n7. RandomForest modeli yapılandırılıyor...")
base_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced_subsample',
    n_jobs=-1,  # Tüm çekirdekleri kullan
    verbose=0,  # Gereksiz çıktıları engelle
    random_state=42
)

# MultiOutputClassifier oluştur
multi_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)


7. RandomForest modeli yapılandırılıyor...


In [13]:
# 7. Entropi tabanlı çok etiketli belirsizlik örneklemesi (paralelleştirilmiş)
def compute_entropy_for_label(proba):
    """Tek bir etiket için entropi hesapla"""
    return -np.sum(proba * np.log2(np.clip(proba, 1e-10, 1.0)), axis=1)

def entropy_based_sampling(model, X_pool, n_instances=10):
    """
    Entropi tabanlı belirsizlik örneklemesi - paralelleştirilmiş versiyon.
    """
    # Modeli kullanarak olasılıkları hesapla
    probas = []
    
    # Paralel olarak her etiket için olasılıkları hesapla
    with parallel_backend('threading', n_jobs=n_cores):
        for estimator in model.estimators_:
            proba = estimator.predict_proba(X_pool)
            probas.append(proba)
    
    # Paralel olarak entropi hesapla
    with parallel_backend('threading', n_jobs=n_cores):
        entropies = Parallel(n_jobs=n_cores)(
            delayed(compute_entropy_for_label)(probas[i]) 
            for i in range(len(probas))
        )
    
    # Tüm etiketlerin entropilerini topla
    total_entropy = np.sum(entropies, axis=0)
    
    # En yüksek entropiye sahip örnekleri seç
    top_indices = np.argsort(total_entropy)[-n_instances:]
    
    return top_indices

In [14]:
# 8. Modelleri ön-eğit ve ara sonuçları kaydet
print("\n8. Modeller ön-eğitiliyor...")

# Dense matrise dönüştür (bazı işlemleri hızlandırmak için)
X_initial_array = X_initial.toarray()

# Modelleri paralel olarak eğit
with parallel_backend('threading', n_jobs=n_cores):
    model = multi_classifier.fit(X_initial_array, y_initial)
    random_model = MultiOutputClassifier(base_classifier, n_jobs=-1).fit(X_initial_array, y_initial)

# Ara sonuçları kaydet
joblib.dump(model, os.path.join(models_dir, 'initial_active_model.joblib'))
joblib.dump(random_model, os.path.join(models_dir, 'initial_random_model.joblib'))
print("Ön-eğitimli modeller başarıyla kaydedildi.")



8. Modeller ön-eğitiliyor...
Ön-eğitimli modeller başarıyla kaydedildi.


In [15]:
# 9. Aktif öğrenme döngüsü
print("\n9. Aktif öğrenme döngüsü başlatılıyor...")

# Kaç tur öğrenme yapılacak ve her turda kaç örnek sorgulanacak
n_queries = 15
n_samples_per_query = 100
active_performance_history = []
random_performance_history = []
active_hamming_history = []
random_hamming_history = []
times_history = {'active_train': [], 'active_predict': [], 'random_train': [], 'random_predict': []}

# İlk performansları hesapla
X_test_array = X_test_tfidf.toarray()

with parallel_backend('threading', n_jobs=n_cores):
    active_initial_preds = model.predict(X_test_array)
    random_initial_preds = random_model.predict(X_test_array)
    
    active_initial_score = f1_score(y_test, active_initial_preds, average='micro')
    random_initial_score = f1_score(y_test, random_initial_preds, average='micro')
    
    active_initial_hamming = hamming_loss(y_test, active_initial_preds)
    random_initial_hamming = hamming_loss(y_test, random_initial_preds)

active_performance_history.append(active_initial_score)
random_performance_history.append(random_initial_score)
active_hamming_history.append(active_initial_hamming)
random_hamming_history.append(random_initial_hamming)

print(f"Başlangıç F1 skoru (Aktif): {active_initial_score:.4f}")
print(f"Başlangıç F1 skoru (Rastgele): {random_initial_score:.4f}")
print(f"Başlangıç Hamming Loss (Aktif): {active_initial_hamming:.4f}")
print(f"Başlangıç Hamming Loss (Rastgele): {random_initial_hamming:.4f}")

# Aktif öğrenme ve rastgele örnekleme döngüleri
X_pool_data = X_pool.toarray()
y_pool_data = y_pool.copy()

# Gereksiz yinelemeleri önlemek için ara sonuçları saklayabiliriz
current_active_train_X = X_initial_array.copy()
current_active_train_y = y_initial.copy()
current_random_train_X = X_initial_array.copy()
current_random_train_y = y_initial.copy()

# Belirsizlik değerlerini de takip edelim
uncertainty_scores = []

for i in tqdm(range(n_queries)):
    
    # AKTIF ÖĞRENME: Entropi tabanlı örnekleme ile en bilgilendirici örnekleri seç
    query_idxs = entropy_based_sampling(model, X_pool_data, n_instances=n_samples_per_query)
    
    # Seçilen örneklerin belirsizlik değerlerini kaydet
    probas = []
    for estimator in model.estimators_:
        proba = estimator.predict_proba(X_pool_data[query_idxs])
        probas.append(proba)
        
    entropies = []
    for j in range(len(probas)):
        entropies.append(compute_entropy_for_label(probas[j]))
    
    total_entropies = np.mean(np.sum(entropies, axis=0))
    uncertainty_scores.append(total_entropies)
    
    # Seçilen örnekleri al
    X_query = X_pool_data[query_idxs]
    y_query = y_pool_data[query_idxs]
    
    # Eğitim verilerini güncelle
    current_active_train_X = np.vstack((current_active_train_X, X_query))
    current_active_train_y = np.vstack((current_active_train_y, y_query))
    
    # RASTGELE ÖRNEKLEME: Aynı sayıda rastgele örnek seç
    random_idxs = np.random.choice(len(X_pool_data), size=n_samples_per_query, replace=False)
    X_random = X_pool_data[random_idxs]
    y_random = y_pool_data[random_idxs]
    
    # Eğitim verilerini güncelle
    current_random_train_X = np.vstack((current_random_train_X, X_random))
    current_random_train_y = np.vstack((current_random_train_y, y_random))
    
    # Modelleri paralel olarak güncelle
    with parallel_backend('threading', n_jobs=n_cores // 2):  # Çekirdekleri bölüştür
        # Aktif model eğitim zamanı
        start_time = time.time()
        model.fit(current_active_train_X, current_active_train_y)
        times_history['active_train'].append(time.time() - start_time)
        
        # Rastgele model eğitim zamanı
        start_time = time.time()
        random_model.fit(current_random_train_X, current_random_train_y)
        times_history['random_train'].append(time.time() - start_time)
    
    # Test performansını paralel olarak ölç
    with parallel_backend('threading', n_jobs=n_cores // 2):
        # Aktif model tahmin zamanı
        start_time = time.time()
        active_preds = model.predict(X_test_array)
        times_history['active_predict'].append(time.time() - start_time)
        
        # Rastgele model tahmin zamanı
        start_time = time.time()
        random_preds = random_model.predict(X_test_array)
        times_history['random_predict'].append(time.time() - start_time)
        
        # Performans metriklerini hesapla
        active_score = f1_score(y_test, active_preds, average='micro')
        random_score = f1_score(y_test, random_preds, average='micro')
        active_hamming = hamming_loss(y_test, active_preds)
        random_hamming = hamming_loss(y_test, random_preds)
    
    active_performance_history.append(active_score)
    random_performance_history.append(random_score)
    active_hamming_history.append(active_hamming)
    random_hamming_history.append(random_hamming)
    
    # Havuzdan seçilen örnekleri çıkar (hem aktif hem de rastgele seçilen)
    all_selected_idxs = np.union1d(query_idxs, random_idxs)
    mask = np.ones(len(X_pool_data), dtype=bool)
    mask[all_selected_idxs] = False
    
    # Havuzu güncelle
    X_pool_data = X_pool_data[mask]
    y_pool_data = y_pool_data[mask]
    
    # İlerlemeyi göster
    print(f"Tur {i+1}/{n_queries}:")
    print(f"  Aktif Öğrenme F1: {active_score:.4f}")
    print(f"  Rastgele Örnekleme F1: {random_score:.4f}")
    print(f"  Fark: {active_score - random_score:.4f}")
    print(f"  Aktif Öğrenme Hamming Loss: {active_hamming:.4f}")
    print(f"  Rastgele Örnekleme Hamming Loss: {random_hamming:.4f}")
    print(f"  Kalan havuz boyutu: {len(X_pool_data)}")
    
    # Her 5 turda bir ara sonuçları kaydet ve görselleştir
    if (i+1) % 5 == 0:
        joblib.dump(active_performance_history, os.path.join(models_dir, f'active_history_tur_{i+1}.joblib'))
        joblib.dump(random_performance_history, os.path.join(models_dir, f'random_history_tur_{i+1}.joblib'))
        joblib.dump(model, os.path.join(models_dir, f'active_model_tur_{i+1}.joblib'))
        joblib.dump(random_model, os.path.join(models_dir, f'random_model_tur_{i+1}.joblib'))
        print(f"  Tur {i+1} ara sonuçları kaydedildi.")
        
        # F1 skor değişimini görselleştir
        plt.figure(figsize=(12, 8))
        plt.plot(range(i+2), active_performance_history, 'b-', marker='o', label='Aktif Öğrenme')
        plt.plot(range(i+2), random_performance_history, 'r--', marker='x', label='Rastgele Örnekleme')
        plt.xlabel('Öğrenme Turu')
        plt.ylabel('F1 Skoru (micro)')
        plt.title(f'Aktif Öğrenme vs. Rastgele Örnekleme Karşılaştırması - Tur {i+1}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(images_dir, f'active_learning_f1_tur_{i+1}.png'))
        plt.close()
        
        # Hamming Loss değişimini görselleştir
        plt.figure(figsize=(12, 8))
        plt.plot(range(i+2), active_hamming_history, 'b-', marker='o', label='Aktif Öğrenme')
        plt.plot(range(i+2), random_hamming_history, 'r--', marker='x', label='Rastgele Örnekleme')
        plt.xlabel('Öğrenme Turu')
        plt.ylabel('Hamming Loss (daha düşük daha iyi)')
        plt.title(f'Aktif Öğrenme vs. Rastgele Örnekleme Hamming Loss - Tur {i+1}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(images_dir, f'active_learning_hamming_tur_{i+1}.png'))
        plt.close()

# Nihai sonuçları kaydet
joblib.dump(active_performance_history, os.path.join(models_dir, 'active_performance_history.joblib'))
joblib.dump(random_performance_history, os.path.join(models_dir, 'random_performance_history.joblib'))
joblib.dump(active_hamming_history, os.path.join(models_dir, 'active_hamming_history.joblib'))
joblib.dump(random_hamming_history, os.path.join(models_dir, 'random_hamming_history.joblib'))
joblib.dump(uncertainty_scores, os.path.join(models_dir, 'uncertainty_scores.joblib'))
joblib.dump(times_history, os.path.join(models_dir, 'times_history.joblib'))
joblib.dump(model, os.path.join(models_dir, 'final_active_model.joblib'))
joblib.dump(random_model, os.path.join(models_dir, 'final_random_model.joblib'))
print("Final sonuçları başarıyla kaydedildi.")



9. Aktif öğrenme döngüsü başlatılıyor...
Başlangıç F1 skoru (Aktif): 0.5765
Başlangıç F1 skoru (Rastgele): 0.5765
Başlangıç Hamming Loss (Aktif): 0.0523
Başlangıç Hamming Loss (Rastgele): 0.0523


  7%|▋         | 1/15 [00:47<11:04, 47.47s/it]

Tur 1/15:
  Aktif Öğrenme F1: 0.5864
  Rastgele Örnekleme F1: 0.5821
  Fark: 0.0042
  Aktif Öğrenme Hamming Loss: 0.0515
  Rastgele Örnekleme Hamming Loss: 0.0521
  Kalan havuz boyutu: 8676


 13%|█▎        | 2/15 [01:34<10:13, 47.18s/it]

Tur 2/15:
  Aktif Öğrenme F1: 0.5932
  Rastgele Örnekleme F1: 0.5955
  Fark: -0.0024
  Aktif Öğrenme Hamming Loss: 0.0513
  Rastgele Örnekleme Hamming Loss: 0.0510
  Kalan havuz boyutu: 8477


...

Tur 13/15:
  Aktif Öğrenme F1: 0.6299
  Rastgele Örnekleme F1: 0.6233
  Fark: 0.0066
  Aktif Öğrenme Hamming Loss: 0.0498
  Rastgele Örnekleme Hamming Loss: 0.0491
  Kalan havuz boyutu: 6291


 93%|█████████▎| 14/15 [13:13<01:06, 66.23s/it]

Tur 14/15:
  Aktif Öğrenme F1: 0.6310
  Rastgele Örnekleme F1: 0.6340
  Fark: -0.0030
  Aktif Öğrenme Hamming Loss: 0.0495
  Rastgele Örnekleme Hamming Loss: 0.0479
  Kalan havuz boyutu: 6095
Tur 15/15:
  Aktif Öğrenme F1: 0.6417
  Rastgele Örnekleme F1: 0.6386
  Fark: 0.0032
  Aktif Öğrenme Hamming Loss: 0.0488
  Rastgele Örnekleme Hamming Loss: 0.0468
  Kalan havuz boyutu: 5896
  Tur 15 ara sonuçları kaydedildi.


100%|██████████| 15/15 [14:24<00:00, 57.60s/it]


Final sonuçları başarıyla kaydedildi.


In [16]:
# 10. Sonuçları görselleştir
plt.figure(figsize=(12, 8))
plt.plot(range(n_queries + 1), active_performance_history, 'b-', marker='o', label='Aktif Öğrenme')
plt.plot(range(n_queries + 1), random_performance_history, 'r--', marker='x', label='Rastgele Örnekleme')
plt.xlabel('Öğrenme Turu')
plt.ylabel('F1 Skoru (micro)')
plt.title('Aktif Öğrenme vs. Rastgele Örnekleme F1 Skor Karşılaştırması')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(images_dir, 'aktif_ogrenme_f1_karsilastirma.png'))
plt.close()

plt.figure(figsize=(12, 8))
plt.plot(range(n_queries + 1), active_hamming_history, 'b-', marker='o', label='Aktif Öğrenme')
plt.plot(range(n_queries + 1), random_hamming_history, 'r--', marker='x', label='Rastgele Örnekleme')
plt.xlabel('Öğrenme Turu')
plt.ylabel('Hamming Loss (daha düşük daha iyi)')
plt.title('Aktif Öğrenme vs. Rastgele Örnekleme Hamming Loss Karşılaştırması')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(images_dir, 'aktif_ogrenme_hamming_karsilastirma.png'))
plt.close()

# Örnek seçimindeki belirsizlik değerlerini görselleştir
plt.figure(figsize=(12, 8))
plt.plot(range(1, n_queries + 1), uncertainty_scores, 'm-', marker='o')
plt.xlabel('Öğrenme Turu')
plt.ylabel('Ortalama Entropi (Belirsizlik)')
plt.title('Aktif Öğrenme Sürecinde Seçilen Örneklerin Belirsizlik Değerleri')
plt.grid(True)
plt.savefig(os.path.join(images_dir, 'aktif_ogrenme_belirsizlik.png'))
plt.close()

# Eğitim ve tahmin sürelerini görselleştir
plt.figure(figsize=(15, 10))
plt.subplot(2, 1, 1)
plt.plot(range(1, n_queries + 1), times_history['active_train'], 'b-', marker='o', label='Aktif Öğrenme')
plt.plot(range(1, n_queries + 1), times_history['random_train'], 'r--', marker='x', label='Rastgele Örnekleme')
plt.xlabel('Öğrenme Turu')
plt.ylabel('Eğitim Süresi (saniye)')
plt.title('Eğitim Süreleri Karşılaştırması')
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(range(1, n_queries + 1), times_history['active_predict'], 'b-', marker='o', label='Aktif Öğrenme')
plt.plot(range(1, n_queries + 1), times_history['random_predict'], 'r--', marker='x', label='Rastgele Örnekleme')
plt.xlabel('Öğrenme Turu')
plt.ylabel('Tahmin Süresi (saniye)')
plt.title('Tahmin Süreleri Karşılaştırması')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(images_dir, 'aktif_ogrenme_sureler.png'))
plt.close()


In [17]:
# 11. Geleneksel RandomForest ile karşılaştır
# Tüm eğitim setiyle eğitilmiş RandomForest'in F1 skoru (önceki çalışmadan)
full_rf_f1 = 0.5233  # RandomForest Çok Etiketli Sınıflandırma-1.ipynb'den alınan değer

# Son tur sonrası etiketli örnek sayıları
final_active_samples = len(initial_idx) + n_queries * n_samples_per_query
percentage_of_data = final_active_samples / len(X_train_text) * 100

print("\n----- GENEL DEĞERLENDİRME -----")
print(f"Toplam eğitim seti: {len(X_train_text)} örnek")
print(f"Aktif öğrenmede kullanılan örnek sayısı: {final_active_samples} örnek ({percentage_of_data:.1f}%)")
print(f"Son F1 skoru (Aktif Öğrenme): {active_performance_history[-1]:.4f}")
print(f"Son F1 skoru (Rastgele Örnekleme): {random_performance_history[-1]:.4f}")
print(f"Son Hamming Loss (Aktif Öğrenme): {active_hamming_history[-1]:.4f}")
print(f"Son Hamming Loss (Rastgele Örnekleme): {random_hamming_history[-1]:.4f}")
print(f"Tam Veri ile RandomForest F1 skoru: {full_rf_f1:.4f}")
print(f"Aktif Öğrenmenin tam veriye göre performans artışı: {(active_performance_history[-1]/full_rf_f1*100)-100:.1f}%")


----- GENEL DEĞERLENDİRME -----
Toplam eğitim seti: 10876 örnek
Aktif öğrenmede kullanılan örnek sayısı: 3500 örnek (32.2%)
Son F1 skoru (Aktif Öğrenme): 0.6417
Son F1 skoru (Rastgele Örnekleme): 0.6386
Son Hamming Loss (Aktif Öğrenme): 0.0488
Son Hamming Loss (Rastgele Örnekleme): 0.0468
Tam Veri ile RandomForest F1 skoru: 0.5233
Aktif Öğrenmenin tam veriye göre performans artışı: 22.6%


In [18]:
# 11. Geleneksel RandomForest ile karşılaştır
# Tüm eğitim setiyle eğitilmiş RandomForest'in F1 skoru (önceki çalışmadan)
full_rf_f1 = 0.5233  # RandomForest Çok Etiketli Sınıflandırma-1.ipynb'den alınan değer

# Son tur sonrası etiketli örnek sayıları
final_active_samples = len(initial_idx) + n_queries * n_samples_per_query
percentage_of_data = final_active_samples / len(X_train_text) * 100

print("\n----- GENEL DEĞERLENDİRME -----")
print(f"Toplam eğitim seti: {len(X_train_text)} örnek")
print(f"Aktif öğrenmede kullanılan örnek sayısı: {final_active_samples} örnek ({percentage_of_data:.1f}%)")
print(f"Son F1 skoru (Aktif Öğrenme): {active_performance_history[-1]:.4f}")
print(f"Son F1 skoru (Rastgele Örnekleme): {random_performance_history[-1]:.4f}")
print(f"Son Hamming Loss (Aktif Öğrenme): {active_hamming_history[-1]:.4f}")
print(f"Son Hamming Loss (Rastgele Örnekleme): {random_hamming_history[-1]:.4f}")
print(f"Tam Veri ile RandomForest F1 skoru: {full_rf_f1:.4f}")
print(f"Aktif Öğrenmenin tam veriye göre performans artışı: {(active_performance_history[-1]/full_rf_f1*100)-100:.1f}%")

# Tek etiketli ve çok etiketli örneklerdeki başarıyı karşılaştır
etiket_sayilari_test = y_test.sum(axis=1)
tek_etiketli_idx = np.where(etiket_sayilari_test == 1)[0]
cok_etiketli_idx = np.where(etiket_sayilari_test > 1)[0]

active_preds = model.predict(X_test_array)
random_preds = random_model.predict(X_test_array)

# Tek etiketli örneklerde performans
active_tek_f1 = f1_score(y_test[tek_etiketli_idx], active_preds[tek_etiketli_idx], average='micro')
random_tek_f1 = f1_score(y_test[tek_etiketli_idx], random_preds[tek_etiketli_idx], average='micro')

# Çok etiketli örneklerde performans
active_cok_f1 = f1_score(y_test[cok_etiketli_idx], active_preds[cok_etiketli_idx], average='micro')
random_cok_f1 = f1_score(y_test[cok_etiketli_idx], random_preds[cok_etiketli_idx], average='micro')

print("\n----- ETİKET SAYISINA GÖRE PERFORMANS -----")
print(f"Tek etiketli örneklerde F1 (Aktif): {active_tek_f1:.4f}")
print(f"Tek etiketli örneklerde F1 (Rastgele): {random_tek_f1:.4f}")
print(f"Çok etiketli örneklerde F1 (Aktif): {active_cok_f1:.4f}")
print(f"Çok etiketli örneklerde F1 (Rastgele): {random_cok_f1:.4f}")


----- GENEL DEĞERLENDİRME -----
Toplam eğitim seti: 10876 örnek
Aktif öğrenmede kullanılan örnek sayısı: 3500 örnek (32.2%)
Son F1 skoru (Aktif Öğrenme): 0.6417
Son F1 skoru (Rastgele Örnekleme): 0.6386
Son Hamming Loss (Aktif Öğrenme): 0.0488
Son Hamming Loss (Rastgele Örnekleme): 0.0468
Tam Veri ile RandomForest F1 skoru: 0.5233
Aktif Öğrenmenin tam veriye göre performans artışı: 22.6%

----- ETİKET SAYISINA GÖRE PERFORMANS -----
Tek etiketli örneklerde F1 (Aktif): 0.6123
Tek etiketli örneklerde F1 (Rastgele): 0.6043
Çok etiketli örneklerde F1 (Aktif): 0.6876
Çok etiketli örneklerde F1 (Rastgele): 0.6895


In [19]:
# Etiket sayısına göre performansı görselleştir
plt.figure(figsize=(10, 6))
labels = ['Tek Etiketli', 'Çok Etiketli']
active_scores = [active_tek_f1, active_cok_f1]
random_scores = [random_tek_f1, random_cok_f1]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, active_scores, width, label='Aktif Öğrenme')
rects2 = ax.bar(x + width/2, random_scores, width, label='Rastgele Örnekleme')

ax.set_ylabel('F1 Skoru')
ax.set_title('Etiket Sayısına Göre F1 Skoru Karşılaştırması')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Çubukların üzerine değerleri yaz
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.4f}',
                    xy=(rect.get_x() + rect.get_width()/2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
plt.savefig(os.path.join(images_dir, 'etiket_sayisi_performans.png'))
plt.close()

# Etiket bazında performans analizi
tag_f1_scores = []
tag_names = []

for i, tag in enumerate(tag_columns):
    # Sadece test setinde yeterli örneği olan etiketleri analiz et
    if np.sum(y_test[:, i]) >= 10:
        tag_f1 = f1_score(y_test[:, i], active_preds[:, i])
        tag_f1_scores.append(tag_f1)
        tag_names.append(tag.replace('tag_', ''))

# En iyi ve en kötü performans gösteren etiketleri bul
sorted_idx = np.argsort(tag_f1_scores)
worst_tags = [tag_names[i] for i in sorted_idx[:5]]
worst_scores = [tag_f1_scores[i] for i in sorted_idx[:5]]
best_tags = [tag_names[i] for i in sorted_idx[-5:]]
best_scores = [tag_f1_scores[i] for i in sorted_idx[-5:]]

print("\n----- ETİKET BAZINDA PERFORMANS -----")
print("En iyi performans gösteren 5 etiket:")
for tag, score in zip(best_tags, best_scores):
    print(f"  {tag}: {score:.4f}")

print("\nEn kötü performans gösteren 5 etiket:")
for tag, score in zip(worst_tags, worst_scores):
    print(f"  {tag}: {score:.4f}")

# En iyi ve en kötü etiketlerin performansını görselleştir
plt.figure(figsize=(14, 8))

plt.subplot(1, 2, 1)
plt.barh(best_tags, best_scores, color='green')
plt.xlabel('F1 Skoru')
plt.title('En İyi Performans Gösteren 5 Etiket')
plt.xlim(0, 1)
plt.grid(axis='x')

plt.subplot(1, 2, 2)
plt.barh(worst_tags, worst_scores, color='red')
plt.xlabel('F1 Skoru')
plt.title('En Kötü Performans Gösteren 5 Etiket')
plt.xlim(0, 1)
plt.grid(axis='x')

plt.tight_layout()
plt.savefig(os.path.join(images_dir, 'en_iyi_en_kotu_etiketler.png'))
plt.close()

# Öğrenme eğrisini daha detaylı görselleştir
plt.figure(figsize=(15, 10))

# Eksen 1: F1 Skoru
ax1 = plt.subplot(2, 1, 1)
ax1.plot(range(n_queries + 1), active_performance_history, 'b-', marker='o', label='Aktif Öğrenme')
ax1.plot(range(n_queries + 1), random_performance_history, 'r--', marker='x', label='Rastgele Örnekleme')
ax1.set_ylabel('F1 Skoru (micro)')
ax1.set_title('Öğrenme Eğrisi - F1 Skoru')
ax1.legend()
ax1.grid(True)

# Eksen 2: Hamming Loss
ax2 = plt.subplot(2, 1, 2)
ax2.plot(range(n_queries + 1), active_hamming_history, 'b-', marker='o', label='Aktif Öğrenme')
ax2.plot(range(n_queries + 1), random_hamming_history, 'r--', marker='x', label='Rastgele Örnekleme')
ax2.set_xlabel('Öğrenme Turu')
ax2.set_ylabel('Hamming Loss')
ax2.set_title('Öğrenme Eğrisi - Hamming Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(images_dir, 'ogrenme_egrisi_detayli.png'))
plt.close()

# İstenen örnek sayısına göre öğrenme eğrisi
ornek_sayilari = [n_initial + i * n_samples_per_query for i in range(n_queries + 1)]

plt.figure(figsize=(12, 8))
plt.plot(ornek_sayilari, active_performance_history, 'b-', marker='o', label='Aktif Öğrenme')
plt.plot(ornek_sayilari, random_performance_history, 'r--', marker='x', label='Rastgele Örnekleme')
plt.xlabel('Etiketli Örnek Sayısı')
plt.ylabel('F1 Skoru (micro)')
plt.title('Örnek Sayısına Göre Performans Karşılaştırması')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(images_dir, 'ornek_sayisi_performans.png'))
plt.close()



----- ETİKET BAZINDA PERFORMANS -----
En iyi performans gösteren 5 etiket:
  ruby-on-rails: 0.7040
  ios: 0.7143
  android: 0.7413
  php: 0.7504
  python: 0.8031

En kötü performans gösteren 5 etiket:
  .net: 0.1695
  iphone: 0.4685
  c: 0.4690
  html: 0.4883
  asp.net: 0.4925


<Figure size 1000x600 with 0 Axes>

In [20]:
# Özet bilgiler dosyası
with open(os.path.join(output_dir, 'ozet_rapor.txt'), 'w', encoding='utf-8') as f:
    f.write("ACTIVE LEARNING - ÖZET RAPOR\n")
    f.write("===========================\n\n")
    
    f.write(f"Toplam eğitim seti: {len(X_train_text)} örnek\n")
    f.write(f"Aktif öğrenmede kullanılan örnek sayısı: {final_active_samples} örnek ({percentage_of_data:.1f}%)\n")
    f.write(f"Test seti: {len(X_test_text)} örnek\n\n")
    
    f.write("PERFORMANS METRİKLERİ\n")
    f.write("--------------------\n")
    f.write(f"Son F1 skoru (Aktif Öğrenme): {active_performance_history[-1]:.4f}\n")
    f.write(f"Son F1 skoru (Rastgele Örnekleme): {random_performance_history[-1]:.4f}\n")
    f.write(f"Fark: {active_performance_history[-1] - random_performance_history[-1]:.4f}\n\n")
    
    f.write(f"Son Hamming Loss (Aktif Öğrenme): {active_hamming_history[-1]:.4f}\n")
    f.write(f"Son Hamming Loss (Rastgele Örnekleme): {random_hamming_history[-1]:.4f}\n")
    f.write(f"Fark: {random_hamming_history[-1] - active_hamming_history[-1]:.4f}\n\n")
    
    f.write(f"Tam Veri ile RandomForest F1 skoru: {full_rf_f1:.4f}\n")
    f.write(f"Aktif Öğrenmenin tam veriye göre performans oranı: {(active_performance_history[-1]/full_rf_f1*100):.1f}%\n\n")
    
    f.write("ETİKET SAYISINA GÖRE PERFORMANS\n")
    f.write("-----------------------------\n")
    f.write(f"Tek etiketli örneklerde F1 (Aktif): {active_tek_f1:.4f}\n")
    f.write(f"Tek etiketli örneklerde F1 (Rastgele): {random_tek_f1:.4f}\n")
    f.write(f"Çok etiketli örneklerde F1 (Aktif): {active_cok_f1:.4f}\n")
    f.write(f"Çok etiketli örneklerde F1 (Rastgele): {random_cok_f1:.4f}\n\n")
    
    f.write("EN İYİ PERFORMANS GÖSTEREN 5 ETİKET\n")
    f.write("-------------------------------\n")
    for tag, score in zip(best_tags, best_scores):
        f.write(f"  {tag}: {score:.4f}\n")
    
    f.write("\nEN KÖTÜ PERFORMANS GÖSTEREN 5 ETİKET\n")
    f.write("--------------------------------\n")
    for tag, score in zip(worst_tags, worst_scores):
        f.write(f"  {tag}: {score:.4f}\n")
    
    f.write("\nZAMAN METRİKLERİ\n")
    f.write("--------------\n")
    f.write(f"Ortalama eğitim süresi (Aktif): {np.mean(times_history['active_train']):.2f} saniye\n")
    f.write(f"Ortalama eğitim süresi (Rastgele): {np.mean(times_history['random_train']):.2f} saniye\n")
    f.write(f"Ortalama tahmin süresi (Aktif): {np.mean(times_history['active_predict']):.2f} saniye\n")
    f.write(f"Ortalama tahmin süresi (Rastgele): {np.mean(times_history['random_predict']):.2f} saniye\n")

print("\nTüm işlemler tamamlandı. Sonuçlar ve grafikler 'active_learning_results' dizininde kaydedildi.")


Tüm işlemler tamamlandı. Sonuçlar ve grafikler 'active_learning_results' dizininde kaydedildi.
